## Table 3

A summary of the classification performance metrics
  for the four individual methods
  and the four different classification combination methods
  when the training data set consists of
  only the sources that are in CFHTLS W1 field,
  has spectroscopic labels available from VVDS,
  and has $i < 22$.
  The definition of the metrics is summarized in
  Table 1.
  The bold entries highlight the best performance values
  within each column.}

In [17]:
from __future__ import division, print_function, unicode_literals
import numpy as np

In [18]:
truth_test = np.loadtxt('../../data/truth_test.dat')

In [19]:
# load TPC results
tpc_test = np.loadtxt('../../data/w1_22_0_tpc_test.mlz', unpack=True, usecols=(2,))

In [20]:
# load SOMc results
som_test = np.loadtxt('../../data/w1_22_0_som_test.mlz', unpack=True, usecols=(2,))

In [21]:
# load HBC results
hbc_all = np.loadtxt('../../data/w1_22_0_median.hbc', unpack=True, usecols=(0,))
hbc_cv = hbc_all[:-len(truth_test)]
hbc_test = hbc_all[-len(truth_test):]

In [26]:
wa_test = np.loadtxt('../../data/w1_22_0.flat')
bom_test = np.loadtxt('../../data/w1_22_0.bom')
stack_test = np.loadtxt('../../data/w1_22_0.stack')
bmc_test = np.loadtxt('../../data/w1_22_0.bmc')

In [27]:
# read in FLUX_RADIUS and MAG_i and make a classification
def morph_class(magnitude, half_radius, cut=[0, 25, 1.0, 3.0]):
    point_source = ((magnitude > cut[0]) & (magnitude < cut[1]) &
                    (half_radius > cut[2]) & (half_radius < cut[3]))
    return point_source.astype(np.int)

mag_i_lower = 17
mag_i_upper = 21.0
r_h_lower = 1.4
r_h_upper = 2.8

r_h_test = np.loadtxt('../../data/flux_radius.test.dat')
mag_i_test = np.loadtxt('../../data/mag_i.test.dat')
morph_test = morph_class(mag_i_test, r_h_test, cut=[mag_i_lower, mag_i_upper, r_h_lower, r_h_upper])

In [28]:
# true galaxies classified as stars
morph_gs = ((morph_test == 1) & (truth_test == 0)).sum()
# true galaxies classified as galaxies
morph_gg = ((morph_test == 0) & (truth_test == 0)).sum()
# true stars classified as galaxies
morph_sg = ((morph_test == 0) & (truth_test == 1)).sum()
# true stars classified as stars
morph_ss = ((morph_test == 1) & (truth_test == 1)).sum()

# galaxy completeness
morph_g_comp = morph_gg / (morph_gg + morph_gs)
# galaxy purity
morph_g_pur = morph_gg / (morph_gg + morph_sg)
# star completeness
morph_s_comp = morph_ss / (morph_ss + morph_sg)
# star purity
morph_s_pur = morph_ss / (morph_ss + morph_gs)

In [29]:
all_test = {'TPC': tpc_test,
            'SOM': som_test,
            'HB': hbc_test,
            'Morphology': morph_test,
            'WA': wa_test,
            'BoM': bom_test,
            'Stacking': stack_test,
            'BMC': bmc_test}

from sklearn.metrics import roc_auc_score

auc = {}

for k in all_test.keys():
    auc[k] = "%.4f" % roc_auc_score(truth_test, all_test[k])
    
max_auc = max(auc, key=auc.get)

auc[max_auc] = r"\textbf{%s}" % auc[max_auc]

print(auc)

{u'BMC': u'\\textbf{0.9738}', u'TPC': u'0.9399', u'WA': u'0.9600', u'Stacking': u'0.9442', u'SOM': u'0.8861', u'HB': u'0.9386', u'BoM': u'0.9587', u'Morphology': u'0.8555'}


In [30]:
from sklearn.metrics import mean_squared_error

mse = {}

for k in all_test.keys():
    mse[k] = "%.4f" % mean_squared_error(truth_test, all_test[k])
    
min_mse = min(mse, key=mse.get)

mse[min_mse] = r"\textbf{%s}" % mse[min_mse]

print(mse)

{u'BMC': u'\\textbf{0.0291}', u'TPC': u'0.0511', u'WA': u'0.0536', u'Stacking': u'0.1847', u'SOM': u'0.0989', u'HB': u'0.0760', u'BoM': u'0.1511', u'Morphology': u'0.0397'}


In [31]:
def calc_completeness_purity(truth, classif, mag, p_cut=0.5, bins=np.arange(16, 26, 0.5)):
    '''
    '''
    # true galaxies classified as stars
    gs_bin, _ = np.histogram(mag[(classif > p_cut) & (truth == 0)], bins=bins)
    # true galaxies classified as galaxies
    gg_bin, _ = np.histogram(mag[(classif < p_cut) & (truth == 0)], bins=bins)
    # true stars classified as galaxies
    sg_bin, _ = np.histogram(mag[(classif < p_cut) & (truth == 1)], bins=bins)
    # true stars classified as stars
    ss_bin, _ = np.histogram(mag[(classif > p_cut) & (truth == 1)], bins=bins)

    # galaxy completeness
    g_comp_bin = gg_bin / (gg_bin + gs_bin)
    g_comp_bin[~np.isfinite(g_comp_bin)] = 1
    # galaxy purity
    g_pur_bin = gg_bin / (gg_bin + sg_bin)
    g_pur_bin[~np.isfinite(g_pur_bin)] = 1
    # star completeness
    s_comp_bin = ss_bin / (ss_bin + sg_bin)
    s_comp_bin[~np.isfinite(s_comp_bin)] = 1
    # star purity
    s_pur_bin = ss_bin / (ss_bin + gs_bin)
    s_pur_bin[~np.isfinite(s_pur_bin)] = 1
    
    return g_comp_bin, g_pur_bin, s_comp_bin, s_pur_bin

    
def find_purity_at(truth_test, clf, step=0.001, gc=None, sc=None):
    
    if bool(gc) == bool(sc):
        raise Exception('Specify only one of gp or sp parameter.')

    pbin = np.arange(0, 1, step)
    
    pure_all = np.zeros(len(pbin))
    comp_all = np.zeros(len(pbin))
    
    for i, p in enumerate(pbin):
        
        # true galaxies classified as stars
        gs = ((clf >= p) & (truth_test == 0)).sum()
        # true galaxies classified as galaxies
        gg = ((clf < p) & (truth_test == 0)).sum()
        # true stars classified as galaxies
        sg = ((clf < p) & (truth_test == 1)).sum()
        # true stars classified as stars
        ss = ((clf >= p) & (truth_test == 1)).sum()
    
        if gc is not None:
            if gg == 0 and gg + sg == 0:
                pure_all[i] = 1
            else:
                pure_all[i] = gg / (gg + sg)
            if gg == 0 and gg + gs == 0:
                comp_all[i] = 1
            else:
                comp_all[i] = gg / (gg + gs)
            
        if sc is not None:
            if ss == 0 and ss + sg == 0:
                comp_all[i] = 1
            else:
                comp_all[i] = ss / (ss + sg)
            if ss == 0 and ss + gs == 0:
                pure_all[i] = 1
            else:
                pure_all[i] = ss / (ss + gs)
    
    if gc is not None:
        ibin = np.argmin(np.abs(comp_all - gc))
        return pbin[ibin], pure_all[ibin]
    
    if sc is not None:
        ibin = np.argmin(np.abs(comp_all - sc))
        return pbin[ibin], pure_all[ibin]

In [32]:
g_pur1 = {}

for k in all_test.keys():
    i, j = find_purity_at(truth_test, all_test[k], gc=morph_g_comp, step=0.001)
    g_pur1[k] = "%.4f" % j
    
max_g_pur1 = max(g_pur1, key=g_pur1.get)
g_pur1[max_g_pur1] = r"\textbf{%s}" % g_pur1[max_g_pur1]
print(g_pur1)

{u'BMC': u'\\textbf{0.9696}', u'TPC': u'0.9350', u'WA': u'0.9208', u'Stacking': u'0.9561', u'SOM': u'0.8843', u'HB': u'0.9325', u'BoM': u'0.9658', u'Morphology': u'0.9597'}


In [33]:
s_pur1 = {}

for k in all_test.keys():
    i, j = find_purity_at(truth_test, all_test[k], sc=morph_s_comp, step=0.001)
    s_pur1[k] = "%.4f" % j
    
max_s_pur1 = max(s_pur1, key=s_pur1.get)
s_pur1[max_s_pur1] = r"$\textbf{%s}$" % s_pur1[max_s_pur1]
print(s_pur1)

{u'BMC': u'$\\textbf{0.9862}$', u'TPC': u'0.7060', u'WA': u'0.8818', u'Stacking': u'0.9309', u'SOM': u'0.4316', u'HB': u'0.6911', u'BoM': u'0.9862', u'Morphology': u'0.9666'}


In [34]:
g_pur2 = {}

for k in all_test.keys():
    i, j = find_purity_at(truth_test, all_test[k], gc=0.96, step=0.001)
    g_pur2[k] = "%.4f" % j
    
max_g_pur2 = max(g_pur2, key=g_pur2.get)
g_pur2[max_g_pur2] = r"$\textbf{%s}$" % g_pur2[max_g_pur2]
print(g_pur2)         

{u'BMC': u'$\\textbf{0.9856}$', u'TPC': u'0.9570', u'WA': u'0.9757', u'Stacking': u'0.9664', u'SOM': u'0.9165', u'HB': u'0.9424', u'BoM': u'0.9790', u'Morphology': u'0.9597'}


In [35]:
s_pur2 = {}
for k in all_test.keys():
    i, j = find_purity_at(truth_test, all_test[k], sc=0.25, step=0.001)
    s_pur2[k] = "%.4f" % j
    
max_s_pur2 = max(s_pur2, key=s_pur2.get)
s_pur2[max_s_pur2] = r"\textbf{%s}" % s_pur2[max_s_pur2]
print(s_pur2)

{u'BMC': u'\\textbf{1.0000}', u'TPC': u'0.9747', u'WA': u'0.9815', u'Stacking': u'0.9983', u'SOM': u'0.6263', u'HB': u'0.6918', u'BoM': u'0.9977', u'Morphology': u'0.9666'}


In [36]:
output = r"""\begin{tabular}{l c c c c c c}
  Classifier & AUC & MSE &
  $p_{g}\left(c_g=%.4f\right)$ & $p_{s}\left(c_s=%.4f\right)$ &
  $p_{g}\left(c_g=0.9600\right)$ & $p_{s}\left(c_s=0.2500\right)$ \\
  \hline
  TPC        & %s & %s & %s & %s & %s & %s \\
  SOMc       & %s & %s & %s & %s & %s & %s \\
  HB         & %s & %s & %s & %s & %s & %s \\
  Morphology & %s & %s & %s & %s & %s & %s \\
  WA         & %s & %s & %s & %s & %s & %s \\
  BoM        & %s & %s & %s & %s & %s & %s \\
  Stacking   & %s & %s & %s & %s & %s & %s \\
  BMC        & %s & %s & %s & %s & %s & %s \\
\end{tabular}""" % (morph_g_comp, morph_s_comp,
                    auc['TPC'], mse['TPC'], g_pur1['TPC'], s_pur1['TPC'], g_pur2['TPC'], s_pur2['TPC'],
                    auc['SOM'], mse['SOM'], g_pur1['SOM'], s_pur1['SOM'], g_pur2['SOM'], s_pur2['SOM'],
                    auc['HB'], mse['HB'],  g_pur1['HB'],  s_pur1['HB'],  g_pur2['HB'], s_pur2['HB'],
                    '-', mse['Morphology'], g_pur1['Morphology'], s_pur1['Morphology'], '-', '-',
                    auc['WA'], mse['WA'], g_pur1['WA'], s_pur1['WA'], g_pur2['WA'], s_pur2['WA'],
                    auc['BoM'], mse['BoM'], g_pur1['BoM'], s_pur1['BoM'], g_pur2['BoM'], s_pur2['BoM'],
                    auc['Stacking'], mse['Stacking'], g_pur1['Stacking'], s_pur1['Stacking'], g_pur2['Stacking'], s_pur2['Stacking'],
                    auc['BMC'], mse['BMC'], g_pur1['BMC'], s_pur1['BMC'], g_pur2['BMC'], s_pur2['BMC']
                    )

print(output)

\begin{tabular}{l c c c c c c}
  Classifier & AUC & MSE &
  $p_{g}\left(c_g=0.9964\right)$ & $p_{s}\left(c_s=0.7145\right)$ &
  $p_{g}\left(c_g=0.9600\right)$ & $p_{s}\left(c_s=0.2500\right)$ \\
  \hline
  TPC        & 0.9399 & 0.0511 & 0.9350 & 0.7060 & 0.9570 & 0.9747 \\
  SOMc       & 0.8861 & 0.0989 & 0.8843 & 0.4316 & 0.9165 & 0.6263 \\
  HB         & 0.9386 & 0.0760 & 0.9325 & 0.6911 & 0.9424 & 0.6918 \\
  Morphology & - & 0.0397 & 0.9597 & 0.9666 & - & - \\
  WA         & 0.9600 & 0.0536 & 0.9208 & 0.8818 & 0.9757 & 0.9815 \\
  BoM        & 0.9587 & 0.1511 & 0.9658 & 0.9862 & 0.9790 & 0.9977 \\
  Stacking   & 0.9442 & 0.1847 & 0.9561 & 0.9309 & 0.9664 & 0.9983 \\
  BMC        & \textbf{0.9738} & \textbf{0.0291} & \textbf{0.9696} & $\textbf{0.9862}$ & $\textbf{0.9856}$ & \textbf{1.0000} \\
\end{tabular}


In [37]:
with open('../../tables/metrics_cut.tex', 'w') as f:
    f.write(output)